# Addendum last week

## dicts

When talking about dicts, there's one new important change from python 3.5 to python 3.7, namely that dicts are now officially sorted (by insertion order):

![3.6](figures/changes_3.6.png)

![3.7](figures/changes_3.7.png)

Meaning ordered dicts became a side-effects of CPython's new implementation of dict, and an official *lanuguage feature* in Python 3.7.  

Soooo this code would only coincidentially work on older versions of Python:

In [1]:
my_dict = {'a': 1, 'b': 2}
my_dict['c'] = 4
assert list(my_dict.keys())[2] == 'c'

You could, however, use an OrderedDict if you have to.  

To keep up with new changes to Python, check https://docs.python.org/3.7/whatsnew/3.7.html. Note that Python 3.7 is the newest version shipped with Anaconda so far.

In [2]:
%%bash

python --version

Python 3.7.3


(what are these percent-signs, you ask? - https://ipython.readthedocs.io/en/stable/interactive/magics.html)

In [3]:
%pwd

'/home/chris/Documents/UNI/sem_14/scientific_programming/lectures/week03-Advanced_Programming_with_Python'

In [4]:
%ls

figures/  lecture.ipynb  save_file.txt  test.txt  TODO


## variables and names

A tool to interactively explore memory layout in Python programs: http://pythontutor.com/live.html#mode=edit.
![tool](figures/tool.png)

.
# Advanced Python
In this lecture we look at a lot of language features that make Python special compared to other programming languages. 

In [5]:
import sys
sys.path.append("..")
from utils import count_down

# The Python Data Model

We have already seen math-operators on numbers:

In [6]:
# Standard math operators work as expected on numbers
a = 2
b = 3

print('a + b = ', a + b)
print('a - b = ', a - b)
print('a * b = ', a * b)
print('a ** b = ', a ** b)  # a to the power of b (a^b is a bit-wise XOR!)
print('a / b = ', a / b)
print('a // b = ', a // b)  # Floor division 
print('b % a = ', b % a)    # Modulo operator (divide, return remainder)

a + b =  5
a - b =  -1
a * b =  6
a ** b =  8
a / b =  0.6666666666666666
a // b =  0
b % a =  1


And we've also seen these operators on strings!

In [7]:
print('hello' + 'world')
print('hello' * 3)

helloworld
hellohellohello


So how does Python know *which one of these* it is supposed to use?  
  
  
Under the hood, most Python syntax is just *syntatic sugar* for method calls. 

So if you call...

In [8]:
mylist = [1, 2, 3]
len(mylist)

3

What Python makes of this is actually:

In [9]:
mylist.__len__()

3

So, under the hood of python there's basically this:

In [10]:
def len(obj):
    return obj.__len__()

If you know the methods that are implicitly called by the common syntax you can design objects that beautifully integrate with the language.

Another example:

In [11]:
3 + 3 

6

is in fact doing 

In [12]:
(3).__add__(3)

6

Let's find out how we can make objects that behave nicely with the rest of the language. As an example we look at a class for representing triples of numeric values.

In [13]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3

In [14]:
Triple(1, 2, 3)

The string representation of our class is not really informative. We can fix this by implementing `__repr__`.

In [15]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
Triple(1, 2, 3)

Triple(1, 2, 3)

To make addition between `Triple`s possible, we have to implement `__add__`. We define the addition `Triple`s just as the elementwise addition of the three numbers.  

In [16]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __add__(self, other):
        num1 = self.nums[0] + other.nums[0]
        num2 = self.nums[1] + other.nums[1]
        num3 = self.nums[2] + other.nums[2]
        return Triple(num1, num2, num3)
        
    
a = Triple(1, 2, 3)
b = Triple(2, 3, 4)

Because we implemented `__add__` we can add triples with the plus operator.
The following three expressions are all the same! The first one is the fast way to write it, which 
internally maps to the second, which internally maps to the third!


In [17]:
print(a + b)
print(a.__add__(b))
print(Triple.__add__(a, b))

Triple(3, 5, 7)
Triple(3, 5, 7)
Triple(3, 5, 7)


In [18]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __add__(self, other):
        if isinstance(other, Triple):
            num1 = self.nums[0] + other.nums[0]
            num2 = self.nums[1] + other.nums[1]
            num3 = self.nums[2] + other.nums[2]
            return Triple(num1, num2, num3)
        elif isinstance(other, int):
            return Triple(self.nums[0]+other, self.nums[1]+other, self.nums[2]+other)
        else:
            return NotImplemted
        
    
a = Triple(1, 2, 3)
b = Triple(2, 3, 4)

In [19]:
a + 1

Triple(2, 3, 4)

Now it would be nice to enable addition between triples and scalar numbers by just elementwise adding the scalar to all three triple values. But the expression

In [20]:
1 + Triple(1, 2, 3)

TypeError: unsupported operand type(s) for +: 'int' and 'Triple'

is interpreted as 

In [21]:
int.__add__(1, Triple(1, 2, 3))

NotImplemented

which returns the special value `NotImplemented`. If a binary operaton does not work when called on the first operand does not work, Python tries to invert the order of operands, calling `__radd__` on the other. If this does not work either a `TypeError` is raised. By implementing `__radd__` we can make scalar addition work without changing the behaivor of the `int`s.

In [22]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __add__(self, other):
        if isinstance(other, Triple):
            num1 = self.nums[0] + other.nums[0]
            num2 = self.nums[1] + other.nums[1]
            num3 = self.nums[2] + other.nums[2]
            return Triple(num1, num2, num3)
        elif isinstance(other, int):
            return Triple(self.nums[0]+other, self.nums[1]+other, self.nums[2]+other)
        else:
            return NotImplemted
    
    def __radd__(self, other):
        num1 = self.nums[0] + other
        num2 = self.nums[1] + other
        num3 = self.nums[2] + other
        return Triple(num1, num2, num3)

In [23]:
1 + Triple(1, 2, 3)

Triple(2, 3, 4)

In [24]:
Triple(1, 5, 7) + 2

Triple(3, 7, 9)

## Exercise
Make the `in` operator work on our triples. For that we need to implement `__contains__`. The statement 
```python 
3 in Triple(1, 2, 3)
```

should become `True` after defining `__contains__`.

In [25]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __contains__(self, item):
        return item in self.nums

In [26]:
3 in Triple(1, 2, 3)

True

You can read more about Python's data model at https://docs.python.org/3/reference/datamodel.html.

In [143]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __contains__(self, item):
        return item in self.nums
    
    def __bool__(self, items):
        return any(self.nums) 

This further resolves a question we had from last week:
### Truth value testing
Any object can be tested for truth value, for use in an if or while condition or as operand of the Boolean operations. The following objects are considered false:
* `None`
* `False`
* Zero of numeric types (`0`, `0.0`)
* Empty sequences and collections: `''`, `()`, `[]`, `{}`, `set()`
* Objects of user-defined classes that return 0 for `len(obj)`

![truthynesss](figures/bool.png)

In [28]:
def find_truthyness(var):
    try:
        return var.__bool__()
    except AttributeError:
        try:
            return var.__len__() != 0
        except AttributeError:
            return True

In [29]:
print(find_truthyness(""))
print(find_truthyness("asdf"), '\n')

print(find_truthyness([]))
print(find_truthyness([1, 2]), '\n')

print(find_truthyness(0))  #int.__bool__(0)
print(find_truthyness(1))  #int.__bool__(1)

False
True 

False
True 

False
True


https://docs.python.org/3/reference/datamodel.html#special-method-names

# Iterables and Iterators
Objects that can be used in `for ... in ...` statements are called *iterable*.

In [30]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
    
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __iter__(self):
        return iter(self.nums)
    
my_triple = Triple(1, 2, 3)


for value in my_triple:
    print(value)

1
2
3


The `__iter__` - magic-method is what makes an object iterable. Behind the scenes, the `iter`-function calls this method to get the iterator.

An *iterator* is an object that implements `__next__`. 

In [31]:
class myrange:
    def __init__(self, n):
        self.i = 0
        self.n = n

    def __next__(self):
        if self.i < self.n:
            self.i += 1
            return self.i
        else:
            raise StopIteration()

In [32]:
a = myrange(2)

Usually, you want to make an iterator also iterable by returning itself from `__iter__`. Here an example of how to create your own `range`-function:

In [33]:
class myrange:
    def __init__(self, n):
        self.i = 0
        self.n = n

    def __iter__(self):
        return self

    def __next__(self):
        if self.i < self.n:
            self.i += 1
            return self.i
        else:
            raise StopIteration()

In [34]:
for i in myrange(5):
    print(i)

1
2
3
4
5


Python relies heavily on iterators, and you should use them everytime Python offers them! the following code would be considered *unpythonic*.

In [35]:
a_list = [10, 20, 30]
for i in range(len(a_list)):
    print(a_list[i])

10
20
30


Instead we prefer using the iterator directly.

In [36]:
a_list = [10, 20, 30]
for number in a_list:
    print(number)

10
20
30


In [37]:
a = myrange(5)
next(a)

1

In [38]:
for i in a:
    print(i)

2
3
4
5


# Properties
Other languages often define *getters* and *setters* to restrict access to object attributes. In Python we can add getter and setter logic using `properties`.

In [39]:
class Triple:
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self._nums[0]) + ", " + str(self._nums[1]) + ", " + str(self._nums[2]) + ")"
    
a = Triple(1, 2, 3)

In [40]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
      
    @property
    def nums(self):
        return self._nums

    
a = Triple(1, 2, 3)
a.nums

(1, 2, 3)

By default, we can not assign to attributes that are declared via properties. 

In [41]:
a.nums = 10, 11, 12

AttributeError: can't set attribute

But we can add a setter with another decorator. This is useful for including validation logic.

In [42]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
      
    @property
    def nums(self):
        return self._nums
    
    @nums.setter
    def nums(self, value):
        if len(value) == 3:
            self._nums = value
        else:
            raise ValueError("Three values are required to set the data.")
    
a = Triple(1, 2, 3)
a.nums = (4, 5, 6)
a.nums

(4, 5, 6)

With `properties` we can add getter and setter logic, without having them surface in our objects interfaces. This also means you can start to write 
your classes with plain attributes and add getters and setters later if required.

# Exceptions

In [43]:
import random

In [44]:
a = [1, 2, 3] if random.randint(0,1) else 1

first_val = a[0] #throws an Exception in 50% of cases

TypeError: 'int' object is not subscriptable

In [45]:
a = [1, 2, 3] if random.randint(0,1) else 1

# we can catch that exception! In Java, this is try-catch, in python it's called try-except
try:
    first_val = a[0]
    print("everything worked!")
except Exception as e:
    print(type(e), e)

<class 'TypeError'> 'int' object is not subscriptable


In [46]:
try:
    file_handle = open('test.txt')
except FileNotFoundError as err:
    print('this will be executed if a FileNotFoundError occurs')
    print(err)
finally:
    print('this will be executed whether the try block throws an error or not')
    try:
        file_handle.close()
    except:
        pass


this will be executed whether the try block throws an error or not


In [47]:
# Exceptions will go up through functions if unhandled
def foo():
    try:
        [1, 2][3] #this will cause an IndexError, however as it isn't handled here, the error is thrown upward to the caller
        open('asdf')
    except FileNotFoundError as err:
        print('file not found error')

try:
    foo()
    print("won't be reached")
except IndexError as err:
    print('index error')

index error


In [48]:
# you can catch multiple exceptions in one try-except statement

try:
    [1,2][3]
except Exception:         #it will start chronologically at the first one, looking if this fits....
    print("this will run") 
except IndexError:        #and if it does, it won't execute the others
    print("this won't..")


this will run


In [49]:
# you can catch multiple exceptions in one try-except statement

try:
    [1,2][3]
except (AttributeError, IndexError):         #it will start chronologically at the first one, looking if this fits....
    print("either atttr or ind") 

either atttr or ind


![exceptions](figures/errors.png)

In [50]:
try:
    while True:
        pass
except KeyboardInterrupt:
    print("I gracefully stopped!")

I gracefully stopped!


`try-except` also has an 'else', which runs if no error was thrown.

In [51]:
try:
    randval = random.randint(0,2)
    print("randval is:", randval)
    if randval == 0:
        [1,2][3]
    elif randval == 1:
        5/0
except IndexError: 
    print("this will run if randval was 0") 
except ZeroDivisionError:       
    print("this will run if randval was 1")
else:
    print("this will run if randval was 2")


randval is: 2
this will run if randval was 2


In [52]:
# You can even extend Exception yourself, to throw your own Exceptions!

class NotTheValueIWantedException(Exception):
    pass

print(isinstance(NotTheValueIWantedException(), Exception))

True


In [53]:
def my_method(value):
    if value != 42 and value != 1337:
        raise NotTheValueIWantedException
        
for i in range(2000):
    try:
        my_method(i)
        print("A value it accepted was:", i)
    except NotTheValueIWantedException:
        pass

A value it accepted was: 42
A value it accepted was: 1337


Remember our Principle? This is because Python is so good and fast at throwing exceptions!
![Glossary: EAFP](../week02-Basic_Programming_with_Python/eafp.png "Glossary: EAFP")

from https://docs.python.org/3/glossary.html

In [54]:
lst = [1, 2, 3, 4, 5]

def even_odd(num):
    return "even" if num % 2 == 0 else "odd"

dct = {}
for elem in lst:
    try:
       dct[even_odd(elem)].append(elem) 
    except KeyError:
        dct[even_odd(elem)] = [elem]
        
print(dct)

{'odd': [1, 3, 5], 'even': [2, 4]}


# Duck Typing

> *"If it looks like a duck and quacks like a duck, it probably is a duck"*.

We stated in lecture 2, that the type of a variable is only checked at the last possible minute. In fact, the philosophy of **duck typing** is that it doesn't even matter what type a variable is -- the only thing that matters is if you can do what you need to with it.

In [55]:
class Animal:
    def is_living():
        return True
    
class LandAnimal(Animal):
    
    def __init__(self):
        self.has_legs = True
        
    def walk(self):
        return "tap tap"
    
class WaterAnimal(Animal):
    def __init__(self):
        self.has_legs = False
    
    def swim(self):
        return "splash"

In [56]:
def move_forward(animal):
    if isinstance(animal, LandAnimal):
        print(animal.walk())
    if isinstance(animal, WaterAnimal):
        print(animal.swim())

In [57]:
animal = LandAnimal() if random.randint(0,1) else WaterAnimal()

move_forward(animal)

splash


In [58]:
class DuckLikeAnimal(LandAnimal, WaterAnimal):
    pass

move_forward(DuckLikeAnimal())

tap tap
splash


![Glossary: Duck Typing](figures/ducktyping.png "Glossary: Duck Typing")

In [59]:
class DuckLikeAnimal(LandAnimal, WaterAnimal):
    def __init__(self, *args, **kwargs):
        self.looks_like = "duck"
        self.quacks_like = "duck"
        super().__init__(*args, **kwargs)    

In [60]:
duck_like = DuckLikeAnimal()

if duck_like.looks_like == "duck" and duck_like.quacks_like == "duck":
    print("For all that matters, it is a duck!")

For all that matters, it is a duck!


So making our animal move *the pythonic way* would include our principle of duck typing togehter with our EAFP principle:

In [61]:
animal = DuckLikeAnimal()

try:
    print(animal.walk())
except AttributeError:
    print(animal.swim())

tap tap


# Factory methods
Sometimes we want to be able to initialize our objects in different ways. A classic pattern of object oriented programming
is the `Factory`. A `Factory`'s only purpose is to initialize other objects. In Python, we do not really need this
pattern, but instead we can use *factory methods* to initialize our object in different ways. For that we can use
`classmethods` that take the class instead of the instance as a first argument.

In [62]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
    
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    @property
    def nums(self):
        return self._nums
    
    @classmethod
    def from_value(cls, num):
        return cls(num, num, num)

Triple.from_value(3)

Triple(3, 3, 3)

# Generators

You remember "enumerate":

In [63]:
grades = ["Outstanding", "Exceeds Expectations", "Acceptable", "Poor", "Dreadful", "Troll"]
for i, grade in enumerate(grades):
    print("num:",i+1,"grade:",grade)

num: 1 grade: Outstanding
num: 2 grade: Exceeds Expectations
num: 3 grade: Acceptable
num: 4 grade: Poor
num: 5 grade: Dreadful
num: 6 grade: Troll


In [64]:
list(enumerate(grades))

[(0, 'Outstanding'),
 (1, 'Exceeds Expectations'),
 (2, 'Acceptable'),
 (3, 'Poor'),
 (4, 'Dreadful'),
 (5, 'Troll')]

If you iterate over a list, you always have to have the full list in memory, which is incredibly inefficient.

In [65]:
for num, elem in enumerate(range(39999999)):
    print(elem)
    if num > 10:
        break

0
1
2
3
4
5
6
7
8
9
10
11



A Python generator function is a function which returns a generator. Generator functions a implicitly defined by the use of `yield` in the function body. `yield` may be used with a value, in which case that value is treated as the "generated" value. The next time `next()` is called on the generator (i.e. in the next step in a for loop, for example), the generator resumes execution from where it called `yield`, not from the beginning of the function. All of the state, like the values of local variables, is recovered and the generator contiues to execute until the next call to `yield`. 

https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

In [66]:
def generate_numbers():
    yield 1
    yield 10
    yield 3
    yield 5
    
for i in generate_numbers():
    print(i)

1
10
3
5


In [67]:
a = generate_numbers()
print(a)

print(next(a))
print()


<generator object generate_numbers at 0x7fe191844840>
1



In [68]:
for i in a:
    print(i)
    
print(next(a)) #will throw a StopIteration

10
3
5


StopIteration: 

When we call a normal Python function, execution starts at function's first line and continues until a return statement, exception, or the end of the function is encountered. 
Once a function returns control to its caller, any work done by the function and stored in local variables is lost. A new call to the function creates everything from scratch. 

A **generator** is a certain kind of function (recognized by the keyword *yield* in place of *return*), that doesn't lose its data. If a generator is called, it will run until the next occurence of the `yield` keyword. When called again, it starts right after that, and runs until the next occurence of `yield`.

A generator is an iterator, which means you can loop over it, call next(), and use it the way you'd use any other iterator

In [69]:
hasattr(a, '__iter__'), hasattr(a, '__next__')

(True, True)

Generators are a perfect way to get rid of too convolutedly nested for-loops:

In [70]:
nested_list = [[[1, 2, 3], [4, 5, 6]],[[7, 8, 9], [10, 11, 12]]]

In [71]:
for i in nested_list:
    for j in i:
        for k in j:
            print(k)

1
2
3
4
5
6
7
8
9
10
11
12


In [72]:
def nested_list_iterator(thelist):
    for i in thelist:
        for j in i:
            for k in j:
                yield k
                
for i in nested_list_iterator(nested_list):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12


Also, generators are perfect if you have complex stuff to loop over and/or want to be able to simply replace that thing you're looping over:

## Exercise
Use a generator to produce even numbers infinitely. Then print the first ten even numbers.

Use a `while True` loop to produces number infinitely. Then wrap the generator function in `enumerate` and `break` after the first ten values. 

In [75]:
def even_numbers():
    i = 0
    while True:
        if i % 2 == 0:
            yield i
        
        i += 1

for i, num in enumerate(even_numbers()):
    print(num)
    if i >= 10:
        break

0
2
4
6
8
10
12
14
16
18
20


In [76]:
def even_numbers():
    i = 0
    while True:
        yield i        
        i += 2

for i, num in enumerate(even_numbers()):
    print(num)
    if i >= 10:
        break

0
2
4
6
8
10
12
14
16
18
20


So a generator is a function that remembers its state in between calls. It's basically the same as this:

In [77]:
class EvenNumberGenerator():
    def __init__(self):
        self.index = 0
    
    def __call__(self):
        self.index += 2
        return self.index
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.__call__()
        
numgen = EvenNumberGenerator()

In [78]:
numgen()

2

In [79]:
for i, num in enumerate(numgen):
    print(num)
    if i >= 10:
        break

4
6
8
10
12
14
16
18
20
22
24


# Context managers (and IO)
Context managers can be seen as conceptual counterpart to functions. While a function presents a chunk of code that is reused in between other operations, a context manager is a chunk of code that is reused *around* other operations.

In [80]:
class PrintingContext:
    
    def __enter__(self):
        print('Entering context.')
    
    def __exit__(self, exception_type, exception_value, traceback):
        print('Exiting context.')
        
with PrintingContext():
    print('I am inside the context')
    
print("I am outside!")

Entering context.
I am inside the context
Exiting context.
I am outside!


In [81]:
def my_func():
    with PrintingContext():
        print("I am inside!")
        return
    
print("before...")
my_func()
print("after!")

before...
Entering context.
I am inside!
Exiting context.
after!


## File IO

Writing to a file.

In [82]:
string = """hello world!
this is chris, and I am writing 
this message!
"""
fh = open('test.txt', 'w') # open needs as arguments the file-path, and a mode ("r": read, "w": write, "a": append, 
                           #                                                    "rb": read binary, "wb": write binary, "a": append binary) 
                           # and returns a file-handle we can work with
fh.write(string)
fh.close()                 # don't forget to close the file afterwards!

In [83]:
%%bash
cat test.txt

hello world!
this is chris, and I am writing 
this message!


Now we can read from the file what we just wrote in there.

In [84]:
# reading example:
fh = open('test.txt', 'r')
lines = fh.readlines()

for line in lines:
    print(line, end='')
    
fh.close()

hello world!
this is chris, and I am writing 
this message!


## Better done with context managers
Context managers are really useful for handling resources that need to released after they are no longer used. The prototypical example is file IO.

In [85]:
with open('save_file.txt', mode='w') as file_context:    # __enter__ is called here.
    file_context.write('You cannot forget to close me.')
# __exit__ is called here.

In [86]:
class File():

    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.open_file = open(self.filename, self.mode)
        return self.open_file

    def __exit__(self, *args):
        self.open_file.close()

with File('save_file.txt', mode='r') as fh:
    print(fh.readlines())

['You cannot forget to close me.']


more info on context managers: https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/

# Lambda Expressions

Lambda expressions can be used to create "small", "throw-away", anonymous functions.

In [87]:
def square_number(x):
    return x**2

square_number(8), type(square_number)

(64, function)

In [88]:
lambda x: x**2

<function __main__.<lambda>>

In [89]:
square_number = lambda x: x**2

square_number(8), type(square_number)

(64, function)

In [90]:
calc_sum = lambda x, y: x + y
calc_sum(2, 3)

5

You can use lambda-expressions for small pieces of code:

In [91]:
now = lambda: pd.to_datetime(datetime.datetime.now()).tz_localize('UTC').tz_convert('Europe/Berlin')
maketime = lambda x: datetime.datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M')
imsave = lambda fname, img: plt.imsave(fname, img, vmin=0, vmax=1)

## Controlling list operations with lambdas

In [92]:
unsorted_list = [6, 1, 45, 67, 3, 7]

# two ways to sort:
new_list = sorted(unsorted_list) # creates a new sorted one, old one stays the same
unsorted_list.sort()             # sorts in-place, the old one will change

print(new_list)
print(unsorted_list)

[1, 3, 6, 7, 45, 67]
[1, 3, 6, 7, 45, 67]


Sorting in  descending order.

In [93]:
unsorted_list.sort(reverse=True) 
unsorted_list

[67, 45, 7, 6, 3, 1]

Sorting according to specific rules can be done with lambda functions. For example you can sort people by their age.

In [94]:
people = [
    {'name': 'Aaron', 'age': 40},
    {'name': 'Berta', 'age': 20},
    {'name': 'Chris', 'age': 29},
]

In [95]:
people.sort(key=lambda item: item['age'])
people

[{'age': 20, 'name': 'Berta'},
 {'age': 29, 'name': 'Chris'},
 {'age': 40, 'name': 'Aaron'}]

or by their name.

In [96]:
people.sort(key=lambda item: item['name'])
people

[{'age': 40, 'name': 'Aaron'},
 {'age': 20, 'name': 'Berta'},
 {'age': 29, 'name': 'Chris'}]

Other functions work similarly, For example you can use the `key` argument in `max`

In [97]:
max(people, key=lambda x: x['age'])

{'age': 40, 'name': 'Aaron'}

## Exercise
Use the `min` function with a `key` argument to find the person that comes first in the alphabet.

In [98]:
min(people, key=lambda x: x['name'])

{'age': 40, 'name': 'Aaron'}

## Map, Filter & Reduce

Python has many features that originally stem from different programming paradigmns. One of these is that of functional programming, where the concept of *map*, *filter*, and *reduce* come from. These functions are there to apply a function to a collection of data.

### Map

Map takes a function and a collection, and simply applys the function to every element of the collection:

In [99]:
items = [1, 2, 3, 4, 5]
squared = list(map(lambda x: x**2, items))
squared

[1, 4, 9, 16, 25]

..which is the same as

In [100]:
items = [1, 2, 3, 4, 5]
squared = []
for i in items:
    squared.append(i**2)

### Filter

`filter` takes a collection and a function that returns a boolean value. As the name suggests, it thus filters the list: it creates a list of elements for which the function returns true.

In [101]:
number_list = range(-5, 5)
less_than_zero = list(filter(lambda x: x < 0, number_list))
less_than_zero

[-5, -4, -3, -2, -1]

### Reduce 

is a really useful function for performing some computation on a list and returning the result. It applies a rolling computation to sequential pairs of values in a list. For example, if you wanted to compute the product of a list of integers.

In [103]:
from functools import reduce #reduce is not in pythons standardlib and must be imported!
mysum = reduce(lambda x,y: x+y, [47,11,42,13])
mysum

113

![tool](figures/reduce_diagram.png)

...which is the same as:

In [104]:
product = 1
thelist = [1, 2, 3, 4]
for num in thelist:
    product = product * num

In [105]:
# real-life example:
def get_newest_commit(commitlist):
    date = lambda x: parser.parse(x).strftime("%s")
    newer = lambda x,y: x if date(x) > date(y) else y
    return reduce(newer, commitlist)

In [106]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


https://www.python-course.eu/lambda.php

# Comprehensions
Comprehensions are a special syntax that simplify the creation of collections.

## List comprehension

To get a list of of squares from a list of numbers we would usually have to write code like this.

In [107]:
original_numbers = [1, 2, 3, 4, 5]
squared_numbers = []
for i in original_numbers:
    squared_numbers.append(i**2)
    
squared_numbers

[1, 4, 9, 16, 25]

However, with a comprehension, we can greatly simplify this.

In [108]:
squared_numbers = [i**2 for i in original_numbers]
squared_numbers

[1, 4, 9, 16, 25]

The general syntax for a comprehension is `[`expression `for` element `in` iterable \[`if` filter_condition\] `]`. Let's see this in action.

In [109]:
original_values = [(1, True), (2, False), (3, False), (4, True), (5, False), (7, True)]
only_trues = []
for i in original_values:
    if i[1]:
        only_trues.append(i[0])

only_trues

[1, 4, 7]

In [110]:
only_trues = [
    i[0]                      # what to do with the values from the old list
    for i in original_values  # for-loop like syntax
    if i[1]                   # filtering. 
]
only_trues

[1, 4, 7]

If we do not just want to filter, but instead do something else with the values that do not satisfy our filter condition, we can use a 
ternary expression.

In [111]:
only_trues_or_zero = [
    i[0] if i[1] else 0       # what to do with the values from the old list
    for i in original_values  # for-loop like syntax
]
only_trues_or_zero

[1, 0, 0, 4, 0, 7]

## Dictionary Comprehension

In [112]:
numbers_and_their_squares = {num: num ** 2 for num in [1,2,3,4,5]}
numbers_and_their_squares

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25}

## Generator Comprehension
Generator comprehension is a compact way to write down generators

In [142]:
a = (i for i in range(10))
print(a) # it's a generator!
next(a)
print(next(a))
print(list(a))

<generator object <genexpr> at 0x7fe191844840>
1
[2, 3, 4, 5, 6, 7, 8, 9]


In [114]:
def scream(*strings, **kwargs):
    print(*[i.upper() for i in strings], **kwargs)

In [115]:
scream("hello", "I am ", "Chris")

HELLO I AM  CHRIS


# Strings and format-strings

## some string-operations

In [116]:
# remove whitespaces at beginning and end
s = "   hello    "
new_s = s.strip()

# split string in multiple parts (gives a list)
s = "test, hello, world"
splitted = s.split(', ')
print(splitted, type(splitted))

# create a string again by joining a list of partial strings
', '.join(splitted) # -> "test, hello, world"

['test', 'hello', 'world'] <class 'list'>


'test, hello, world'

## Format-strings: version 1

Standard format-strings are C-style. The `%`-operator formats a set of varaibles in a tuple together with a special format-strings. This syntax is still from Python 2 and there is no good reason to still use it today.

In [117]:
import math

an_int = 15
a_float = math.pi #3.141592653589793
a_string = "string!"

string = "An int: %d, a rounded float: %.2f, a string: %s" % (an_int, a_float, a_string)
print(string)

An int: 15, a rounded float: 3.14, a string: string!


## Format-strings: version 2

Python's native format()-method is far more powerful than the borrowed c-syntax. For a complete list of it's arguments, look at https://docs.python.org/3.4/library/string.html#formatspec. For a nice small overview, have a look at https://wiki.python.org/moin/FormatReference

In [118]:
people = [
    {'name': 'aaron', 'age': 40},
    {'name': 'berta', 'age': 20},
    {'name': 'chris', 'age': 21},
]

# the {} are placeholders and are filled with the arguments of format()
for person in people:
    print("{} is {} years old!".format(person['name'], person['age']))

aaron is 40 years old!
berta is 20 years old!
chris is 21 years old!


The `{}` placeholders can be given names to make order irrelevant.

In [119]:
for person in people:
    print("{name} is {age} years old!".format(
        age=person['age'],
        name=person['name']))

aaron is 40 years old!
berta is 20 years old!
chris is 21 years old!


Names allow for neat dict-unpacking into the `format()` method.

In [120]:
for person in people:
    print("{name} is {age} years old!".format(**person))

aaron is 40 years old!
berta is 20 years old!
chris is 21 years old!


In [121]:
# format() allows for many many formatting options, such as justification and
# conversion to decimal places etc...
nums = [10**i for i in range(5)] # -> [1, 10, 100, 1000, 10000]

for n in nums:
    # >     right justified
    # 10    10 characters long
    # .2    show 2 decimal places
    # f     display as floating point number
    print("{:>10.2f}".format(n))

      1.00
     10.00
    100.00
   1000.00
  10000.00


## Format-strings: version 3

Since Python 3.6, there is nice super-easy way to produce formatted string literals: `fstrings`! `fstrings` are very readable and should be the preferred way to format strings, except if you need to do something very complex that requires `str.format`.

In [122]:
name = "Fred"
string = f"He said his name is {name}."
string

'He said his name is Fred.'

In [123]:
f"12 + 16 = {12 + 16}"

'12 + 16 = 28'

In [124]:
width = 10
precision = 5
value = 12.34567
f"result: {value:{width}.{precision}}" 

'result:     12.346'

# Decorators
Decorators a functions that change the functionality of other functions or classes. This should be usually be done in a transparent manner, i.e. the interface of the original function stays the same, while the functionaliy is added around it.

In [125]:
def substract(x, y):
    return x - y

def decorated_substract(*args, **kwargs):
    print('~~~ result of', substract.__name__, '~~~')
    result = substract(*args, **kwargs)              
    print(result)                               
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')        
    return result    
    
decorated_substract(5, 2);

~~~ result of substract ~~~
3
~~~~~~~~~~~~~~~~~~~~~~~~~~~


This however only creates a new function, containing a changed behaviour of the substract-function. What if we want to change the behaviour of arbitrary functions?

In [126]:
def substract(x, y):
    return x - y

def add(x, y):
    return x + y

def decorated(func, *args, **kwargs):
    result = func(*args, **kwargs)              
    print('~~~ result of', func.__name__, '~~~')
    print(result)                               
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')        
    return result    
    
decorated(add, 5, 2)

~~~ result of add ~~~
7
~~~~~~~~~~~~~~~~~~~~~~~~~~~


7

In [127]:
decorated(substract, 5, 2)

~~~ result of substract ~~~
3
~~~~~~~~~~~~~~~~~~~~~~~~~~~


3

We're still not okay with this, because we want to change the behaviour of `add` itself!

In [128]:
def print_decorator(func):                           # func is the method which will be decorated by this
        
    print("This occurs when we re-define the function")
    
    #if we define function = decorated(function), the new function will be this:
    
    def inner(*args, **kwargs):                      # we define a new function here, taking any parameters...
        result = func(*args, **kwargs)               # which, when called, executes the original function with these parameters...
        print('~~~ result of', func.__name__, '~~~') # prints name of original funciton...
        print(result)                                # prints the result of the function...
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')         # some lines...
        return result                                # and returns that result of that function 
    
    return inner   # the new function is this inner function!

In [129]:
decorated_add = print_decorator(add)

This occurs when we re-define the function


In [130]:
decorated_add(3, 5)

~~~ result of add ~~~
8
~~~~~~~~~~~~~~~~~~~~~~~~~~~


8

In [131]:
add = print_decorator(add)
add(3,5)

This occurs when we re-define the function
~~~ result of add ~~~
8
~~~~~~~~~~~~~~~~~~~~~~~~~~~


8

Python provides a syntax for the assignment `function = decorated(function)`. This however just *syntactic sugar* for calling the decorator directly. 

In [132]:
@print_decorator #multiply = print_decorator(multiply)  
def multiply(x, y):
    return x * y

multiply(3, 5)
multiply(4, 5)

This occurs when we re-define the function
~~~ result of multiply ~~~
15
~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~ result of multiply ~~~
20
~~~~~~~~~~~~~~~~~~~~~~~~~~~


20

Another example:

In [133]:
def bold(fn):
    """wraps the result of a function such that it's bold"""
    def wrapped():
        return "<b>" + fn() + "</b>"
    return wrapped


@bold #hello = bold(hello)
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

'<b>hello world</b>'

In [134]:
from IPython.display import HTML
HTML(hello())

We can even chain decorators!

In [135]:
def bold(fn):
    """wraps the result of a function such that it's bold"""
    def wrapped():
        return "<b>" + fn() + "</b>"
    return wrapped

def italic(fn):
    """wraps the result of a function such that it's italics"""
    def wrapped():
        return "<i>" + fn() + "</i>"
    return wrapped

@bold #hello = bold(hello)
@italic #hello = italic(bold(hello))
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

'<b><i>hello world</i></b>'

In [136]:
hello?

Signature: hello()
Docstring: <no docstring>
File:      ~/Documents/UNI/sem_14/scientific_programming/lectures/week03-Advanced_Programming_with_Python/<ipython-input-135-ec26579624f2>
Type:      function


That's it almost it for basic knowledge of decorators!
There's just one important thing: If we replace the original function by the decorated version, we lose all information of the orginal function, as its docstring, information about arguments, etc. To make up for that, we use *another decorator*, namely `functools.wraps`. This simply copies the docstring of the original function to the new one.

In [137]:
from functools import wraps
from IPython.display import HTML

def html(fn):
    @wraps(fn)
    def wrapped():
        return HTML(fn())
    return wrapped


def bold(fn):
    @wraps(fn)
    def wrapped():
        return "<b>" + fn() + "</b>"
    return wrapped

def italic(fn):
    @wraps(fn)
    def wrapped():
        return "<i>" + fn() + "</i>"
    return wrapped

@html
@bold
@italic
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

In [138]:
hello?

Signature: hello()
Docstring: prints 'hello world'
File:      ~/Documents/UNI/sem_14/scientific_programming/lectures/week03-Advanced_Programming_with_Python/<ipython-input-137-473bdfbbb84c>
Type:      function


## Exercise
Define and apply a decorator that makes a string appear red. You can achieve this by wrapping the string in `<span style='color: red'> str </span>`

In [139]:
def red(fn):
    @wraps(fn)
    def wrapped():
        return "<span style='color: red'>" + fn() + "</span>"
    return wrapped

@html
@red
@bold
@italic
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()